In [20]:
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
from bokeh.models import Legend
import numpy as np

In [21]:
df = pd.read_csv(r'scrapeTest_STRAVA_withDf_1_FINAL_FOR_TESTING.csv')

## Riders with the most leadboard positions

In [22]:
#count each person's occurence 
athlete_counts = df['athlete_id'].value_counts()

#merge series and their respective id in new DF
athlete_info = pd.DataFrame({'athlete_id': athlete_counts.index, 'count': athlete_counts.values})

#merge names with their respective id number
result = pd.merge(athlete_info, df[['athlete_id', 'athlete_name']], on='athlete_id').drop_duplicates()

print(result[['athlete_name', 'count']].head(20))

                        athlete_name  count
0                Mars Cirilli 🇮🇹🇺🇸🇧🇷    140
140                     Andrew Wells    133
273                   David Melendez    114
387                     Akiva Wasser     70
457                       Kelvin Mai     66
523                       Rob Oliver     63
586                     Eric Schmidt     61
647                         Joe Todd     60
707                     Peter Bejzol     59
766                    Azriel Wasser     56
822                   Bogdan Wielgos     53
875                       Bryam Mora     53
928                  Vincent Ribeiro     51
979                   William Hacker     51
1030                   Nolan Montiel     51
1081                George Schroeder     48
1082  George Schroeder - R.I.P. Noel     48
1129                     Aidan Moran     48
1177                  Chris Williams     47
1224                        bg payne     47


## TOP 10 density chart

In [23]:
# Select top 10 athletes whose athlete_id occurs the most
top_10_athletes = df['athlete_id'].value_counts().nlargest(10).index

#include only the top 10 athletes
top_10_df = df[df['athlete_id'].isin(top_10_athletes)]

#map athlete IDs to names with a dictionary
id_to_name = dict(zip(top_10_df['athlete_id'], top_10_df['athlete_name']))

#create Bokeh figure for plotting
p = figure(title='Density Plot of Rank for Top 10 Athletes', x_axis_label='Rank', y_axis_label='Density')

#color palette for each athlete
palette = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

#plot each athlete's density plot
legends = []
for i, (athlete_id, data) in enumerate(top_10_df.groupby('athlete_id')):
    athlete_name = id_to_name[athlete_id]
    hist, edges = np.histogram(data['rank'], density=True, bins=10)
    glyph = p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color=palette[i], line_color=None, alpha=0.5)
    legends.append((athlete_name, [glyph]))

legend = Legend(items=legends, location='top_right')
p.add_layout(legend)
output_file("density_plot.html")
show(p)

## Placement by month

In [24]:
#set column to proper formating for date manipulation 
df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y')

In [25]:
#group by month and count occurrences by month
counts = df.groupby(df["date"].dt.month).date.count()

#define months to match data and their corresponding names for plotting 
x_vals = list(counts.index)
x_names = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

#create Bokeh figure for plotting
p = figure(x_range=x_names, title='Leaderboard Placements by Month', x_axis_label='Month', y_axis_label='Top Leaderboard Placement')
source = ColumnDataSource(data=dict(x=x_names, counts=counts.values))
p.vbar(x='x', top='counts', width=0.5, source=source, color="blue", alpha=0.7)
p.text(x='x', y='counts', text='counts', text_align='center', text_baseline='bottom', text_font_size="10pt", source=source)
p.xaxis.major_label_orientation = "vertical"

output_file("leaderboard_placements_month.html")
show(p)

## Segments with smallest GAPS

This data point will show which routes have the closest average time for top leaderboard placements, this means most people at the top of the leaderboard will have approximately the same time. This is good to know if you want to get on to the leaderboard and want to avoid segments that are harder to get top ranking in.

In [26]:
df_time=df.copy()

#convert times to timedelta
def convert_to_timedelta(time_str):
    if 's' in time_str:
        seconds = int(time_str[:-1])
        return pd.Timedelta(seconds=seconds)
    elif ':' in time_str:
        parts = time_str.split(':')
        minutes = int(parts[0])
        seconds = int(parts[1])
        return pd.Timedelta(minutes=minutes, seconds=seconds)
    else:
        return pd.to_timedelta(time_str)

df_time['time'] = df_time['time'].apply(convert_to_timedelta)

print(df_time[['SegNum', 'time']])

         SegNum            time
0      24478874 0 days 00:09:12
1      24478874 0 days 00:11:03
2      24478874 0 days 00:11:16
3      24478874 0 days 00:13:44
4      24478874 0 days 00:15:19
...         ...             ...
33247  28891234 0 days 00:09:04
33248  28891234 0 days 00:09:09
33249  28891234 0 days 00:09:16
33250  28891234 0 days 00:09:31
33251  28891234 0 days 00:13:07

[33252 rows x 2 columns]


In [27]:
#group by SegNum and calculate the gap in time taken for each segment 
grouped = df_time.groupby('SegNum')['time'].apply(lambda x: x.max() - x.min())

#sort by smalled gaptimes
sorted_routes = grouped.sort_values()

#select top 15 (smallest gaps)
top_15_routes = sorted_routes.head(15)

print(top_15_routes)
#operation that is not shown in file:
#matching with Segment name and the modifcation of how time will be shown e.g. the value "0 days 00:00:03" will be shown as "3 seconds" on map

SegNum
8065205    0 days 00:00:00
13439221   0 days 00:00:00
11238721   0 days 00:00:01
12408926   0 days 00:00:02
1750576    0 days 00:00:02
8646723    0 days 00:00:02
16425679   0 days 00:00:02
9197143    0 days 00:00:02
2734138    0 days 00:00:02
1995199    0 days 00:00:02
613198     0 days 00:00:02
21458728   0 days 00:00:03
3884715    0 days 00:00:03
21367438   0 days 00:00:03
4934816    0 days 00:00:03
Name: time, dtype: timedelta64[ns]
